# Auvix

## Инициализация

In [1]:
import os
import sys

from django.utils import timezone

sys.path.append('/home/ubuntu/anodos.ru/anodos/')
os.environ['DJANGO_SETTINGS_MODULE'] = 'anodos.settings'

from django.core.wsgi import get_wsgi_application
application = get_wsgi_application()


import re
import catalog.runner
from catalog.models import *


class Runner(catalog.runner.Runner):

    name  = 'Auvix'
    alias = 'auvix'
    url = {
        'start'  : 'https://b2b.auvix.ru/',
        'login'  : 'https://b2b.auvix.ru/?login=yes',
        'price'  : 'https://b2b.auvix.ru/prices/Price_AUVIX_dealer_xml.zip'}

    def __init__(self):

        super().__init__()

        self.stock   = self.take_stock('stock',   'склад', 3, 10)
        self.factory = self.take_stock('factory', 'на заказ', 20, 80)

        self.count = {'product': 0, 'party': 0}

    def run(self):

        payload = {
            'AUTH_FORM'     : 'Y',
            'TYPE'          : 'AUTH',
            'backurl'       : '/',
            'USER_LOGIN'    : self.updater.login,
            'USER_PASSWORD' : self.updater.password,
            'Login'         : '%C2%A0',
            'USER_REMEMBER' : 'Y'}
        if self.login(payload):
            print('Авторизован.')
        else:
            print('Не удалось авторизоваться')
            return False

        # Загружаем данные
        self.data = self.load_data(self.url['price'])
        self.data = self.unpack_xml(self.data)
        if self.data is not None:
            print('Данные загружены.')
        else:
            print('Ошибка загрузки данных!')
            return None

s = Runner()

In [10]:
def get_string(element, query):

    try:
        result = element.xpath(query)[0].text.strip()
    except Exception:
        result = ''

    return result

s.get_string = get_string

## Загрузка данных

In [2]:
s.run()

Получены параметры авторизации.


/usr/local/lib/python3.5/dist-packages/requests/packages/urllib3/connectionpool.py:852: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)
/usr/local/lib/python3.5/dist-packages/requests/packages/urllib3/connectionpool.py:852: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)
/usr/local/lib/python3.5/dist-packages/requests/packages/urllib3/connectionpool.py:852: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)
/usr/local/lib/python3.5/dist-packages/requests/packages/urllib3/conne

Авторизован.


/usr/local/lib/python3.5/dist-packages/requests/packages/urllib3/connectionpool.py:852: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)


Данные загружены.


## Готовим данные

In [3]:
print(s.data)

<Element doc at 0x7fb43c61e048>


In [4]:
currency = {
    'USD'   : s.usd,
    'Евро'  : s.eur,
    'Рубль' : s.rub}

In [5]:
# Перенос данных
tree = s.data

## Парсим данные

In [14]:
s.reg = re.compile('\[(?P<article>[0-9A-Za-z\.\-\_ ]+)\]')

products = []
categories = []

for group in tree.xpath('.//Группа'):

    category = s.get_string(group, './Наименование')

    for element in group.xpath('./Товары/Товар'):

        product = {}
        party = {}

        # Производитель
        product['vendor'] = Vendor.objects.get_by_key(s.updater, s.get_string(element, './Производитель'))

        # Продукт
        product['name'] = s.fix_name(s.get_string(element, './Наименование'))

        product['article'] = s.get_string(element, './Модель')
        product['article_alt'] = re.search(s.reg, s.get_string(element, './Наименование'))
        if product['article_alt']:
            product['article'] = product['article_alt'].group('article')
        product['article'] = s.fix_article(product['article'])

        product = Product.objects.take(article  = product['article'],
                                       vendor   = product['vendor'],
                                       name     = product['name'],
                                       unit     = s.default_unit,
                                       category = category)

        if product:
            s.count['product'] += 1
        else:
            continue

        # Партии
        party['article'] = s.fix_article(s.get_string(element, './Артикул'))

        party['quantity'] = s.fix_quantity(s.get_string(element, './Количество'))

        party['currency'] = s.get_string(element, './Валюта')
        if party['currency']:
            party['currency'] = currency[party['currency']]
        else:
            party['currency'] = None

        party['price_in'] = s.fix_price(s.get_string(element, './Цена_3'))
        party['price_out'] = s.fix_price(s.get_string(element, './Цена_1'))

        if party['quantity'] != 0:
            party = Party.objects.make(product    = product,
                                       stock      = s.stock,
                                       price      = party['price_in'],
                                       price_type = s.dp,
                                       currency   = party['currency'],
                                       quantity   = party['quantity'],
                                       unit       = s.default_unit,
                                       time       = s.start_time)
            s.count['party'] += 1

        else:
            party = Party.objects.make(product    = product,
                                       stock      = s.factory,
                                       price      = party['price_in'],
                                       price_type = s.dp,
                                       currency   = party['currency'],
                                       quantity   = None,
                                       unit       = s.default_unit,
                                       time       = s.start_time)
            s.count['party'] += 1

        products.append(product)
        categories.append(category)

InFocus SP1080
InFocus SP1080 = 705,0 $; 1 on auvix-stock
InFocus IN119HDx
InFocus IN119HDx = 757,0 $; 1 on auvix-stock
InFocus IN128HDx
InFocus IN128HDx = 1 039,0 $; 1 on auvix-stock
InFocus IN128HDSTx
InFocus IN128HDSTx = 1 119,0 $; 1 on auvix-stock
InFocus IN2128HDx
InFocus IN2128HDx = 1 183,0 $; 1 on auvix-stock
InFocus IN112x
InFocus IN112x = 345,0 $; 1 on auvix-stock
InFocus IN220
InFocus IN220 = 367,0 $; 1 on auvix-stock
InFocus IN114x
InFocus IN114x = 429,0 $; 20 on auvix-stock
InFocus IN124a
InFocus IN124a = 598,0 $; 1 on auvix-stock
InFocus IN124x
InFocus IN124x = 689,0 $; 1 on auvix-stock
InFocus IN2124a
InFocus IN2124a = 685,0 $; 1 on auvix-stock
InFocus IN2124x
InFocus IN2124x = 1 010,0 $; 1 on auvix-stock
InFocus IN116x
InFocus IN116x = 515,0 $; 1 on auvix-stock
InFocus IN126a
InFocus IN126a = 729,0 $; 2 on auvix-stock
InFocus IN126x
InFocus IN126x = 779,0 $; 1 on auvix-stock
InFocus IN2126a
InFocus IN2126a = 784,0 $; 1 on auvix-stock
InFocus IN2126x
InFocus IN2126x = 1 0

Sony BKM-PJ20
Sony BKM-PJ20 = 71 075,0 р.; None on auvix-factory
Sony IFU-PN250A
Sony IFU-PN250A = 5 015,0 р.; None on auvix-factory
Sony PSS-650
Sony PSS-650 = 105 315,0 р.; None on auvix-factory
Sony PSS-650P
Sony PSS-650P = 24 574,0 р.; None on auvix-factory
Panasonic PT-VX420E
Panasonic PT-VX420E = 59 500,0 р.; 1 on auvix-stock
Panasonic PT-VX425NE
Panasonic PT-VX425NE = 76 500,0 р.; 1 on auvix-stock
Panasonic PT-VW350E
Panasonic PT-VW350E = 63 000,0 р.; None on auvix-factory
Panasonic PT-VW355NE
Panasonic PT-VW355NE = 80 750,0 р.; None on auvix-stock
Panasonic PT-VX600E
Panasonic PT-VX600E = 93 500,0 р.; None on auvix-factory
Panasonic PT-VX605NE
Panasonic PT-VX605NE = 123 000,0 р.; 1 on auvix-stock
Panasonic PT-VW530E
Panasonic PT-VW530E = 89 250,0 р.; None on auvix-factory
Panasonic PT-VW535NE
Panasonic PT-VW535NE = 119 000,0 р.; None on auvix-factory
Panasonic PT-VZ570E
Panasonic PT-VZ570E = 133 875,0 р.; None on auvix-factory
Panasonic PT-VZ575NE
Panasonic PT-VZ575NE = 157 250

Panasonic ET-PKE16B
Panasonic ET-PKE16B = 191 878,0 р.; None on auvix-factory
Panasonic ET-PKE16H
Panasonic ET-PKE16H = 310 470,0 р.; None on auvix-factory
Panasonic ET-PKE16S
Panasonic ET-PKE16S = 174 326,0 р.; None on auvix-factory
Panasonic ET-PKC100W
Panasonic ET-PKC100W = 20 284,0 р.; 10 on auvix-stock
Panasonic ET-PKC200W
Panasonic ET-PKC200W = 11 877,0 р.; 1 on auvix-stock
Panasonic ET-PKD120H
Panasonic ET-PKD120H = 63 427,0 р.; None on auvix-factory
Panasonic ET-ACF100
Panasonic ET-ACF100 = 8 302,0 р.; None on auvix-factory
Panasonic ET-RFE200
Panasonic ET-RFE200 = 6 331,0 р.; None on auvix-factory
Panasonic ET-RFV400
Panasonic ET-RFV400 = 5 992,0 р.; None on auvix-factory
Panasonic ET-EMF300
Panasonic ET-EMF300 = 6 019,0 р.; None on auvix-factory
Panasonic ET-WM200E
Panasonic ET-WM200E = 13 017,0 р.; 1 on auvix-stock
Panasonic ET-UK20
Panasonic ET-UK20 = 45 000,0 р.; None on auvix-factory
Panasonic ET-YFB100G
Panasonic ET-YFB100G = 122 000,0 р.; None on auvix-factory
Panasonic

Panasonic ET-DLE150
Panasonic ET-DLE150 = 152 000,0 р.; None on auvix-factory
Optoma H1Z1D2300012
Optoma H1Z1D2300012 = 404,0 $; 3 on auvix-stock
Panasonic ET-DLE250
Panasonic ET-DLE250 = 152 000,0 р.; None on auvix-factory
Panasonic ET-DLE350
Panasonic ET-DLE350 = 152 000,0 р.; None on auvix-factory
Panasonic ET-DLE450
Panasonic ET-DLE450 = 152 000,0 р.; None on auvix-factory
Optoma H1Z1D2300013
Optoma H1Z1D2300013 = 1 755,0 $; 1 on auvix-stock
NEC NP11FL
NEC NP11FL = 1 896,0 $; None on auvix-factory
NEC NP12ZL
NEC NP12ZL = 1 896,0 $; None on auvix-factory
NEC NP13ZL
NEC NP13ZL = 476,0 $; None on auvix-stock
NEC NP14ZL
NEC NP14ZL = 1 896,0 $; None on auvix-factory
NEC NP15ZL
NEC NP15ZL = 1 896,0 $; None on auvix-factory
NEC NP30ZL
NEC NP30ZL = 1 835,0 $; None on auvix-factory
Optoma H1Z1D2300014
Optoma H1Z1D2300014 = 1 214,0 $; 2 on auvix-stock
Panasonic ET-ELW03
Panasonic ET-ELW03 = 238 400,0 р.; None on auvix-factory
Panasonic ET-ELW06
Panasonic ET-ELW06 = 260 000,0 р.; None on auvi

InFocus SP-LAMP-084 = 248,0 $; None on auvix-factory
InFocus SP-LAMP-085
InFocus SP-LAMP-085 = 248,0 $; None on auvix-factory
InFocus SP-LAMP-086
InFocus SP-LAMP-086 = 225,0 $; None on auvix-factory
InFocus SP-LAMP-087
InFocus SP-LAMP-087 = 248,0 $; None on auvix-factory
InFocus SP-LAMP-088
InFocus SP-LAMP-088 = 309,0 $; None on auvix-factory
InFocus SP-LAMP-089
InFocus SP-LAMP-089 = 225,0 $; None on auvix-factory
InFocus SP-LAMP-090
InFocus SP-LAMP-090 = 464,0 $; None on auvix-factory
InFocus SP-LAMP-091
InFocus SP-LAMP-091 = 217,0 $; None on auvix-factory
InFocus SP-LAMP-092
InFocus SP-LAMP-092 = 293,0 $; None on auvix-stock
InFocus SP-LAMP-093
InFocus SP-LAMP-093 = 225,0 $; None on auvix-factory
InFocus SP-LAMP-094
InFocus SP-LAMP-094 = 281,0 $; None on auvix-factory
NEC GT60LP
NEC GT60LP = 777,0 $; None on auvix-factory
NEC GT60LPS
NEC GT60LPS = 777,0 $; None on auvix-factory
NEC MT820/1020
NEC MT820/1020 = 454,0 $; None on auvix-factory
NEC MT50LP
NEC MT50LP = 567,0 $; None on auv

...

Global Cache GC-SP1
Global Cache GC-SP1 = 30,0 $; 13 on auvix-stock
Global Cache GC-SС1
Global Cache GC-SС1 = 30,0 $; 2 on auvix-stock
Global Cache GC-SV1
Global Cache GC-SV1 = 48,0 $; 1 on auvix-stock
Global Cache GC-IRE
Global Cache GC-IRE = 72,0 $; 3 on auvix-stock
Global Cache GC-BL2
Global Cache GC-BL2 = 72,0 $; 3 on auvix-stock
Global Cache GC-RG1
Global Cache GC-RG1 = 72,0 $; 3 on auvix-stock
Crestron PW-2410RU
Crestron PW-2410RU = 297,0 $; 1 on auvix-stock
Crestron CH-HRMOT1A-IW
Crestron CH-HRMOT1A-IW = 154,0 $; None on auvix-factory
Kramer Electronics FC-10ETH
Kramer Electronics FC-10ETH = 11 310,0 р.; None on auvix-factory
Crestron CEN-IDOCV-DSW-W-T
Crestron CEN-IDOCV-DSW-W-T = 840,0 $; None on auvix-factory
Crestron ABARI-1
Crestron ABARI-1 = 115,0 $; 1 on auvix-stock
Crestron CEN-IDOCV-B-S
Crestron CEN-IDOCV-B-S = 840,0 $; 2 on auvix-stock
Crestron CEN-IDOCV-DSW-B-T
Crestron CEN-IDOCV-DSW-B-T = 840,0 $; 1 on auvix-stock
Crestron CEN-IDOCV-W
Crestron CEN-IDOCV-W = 840,0 $; N

Bosch DCN-WDCS-D
Bosch DCN-WDCS-D = 71 562,0 р.; None on auvix-factory
Bosch DCN-WDDCS-D
Bosch DCN-WDDCS-D = 90 376,0 р.; None on auvix-factory
Bosch F01U066789
Bosch F01U066789 = 8 505,0 р.; 20 on auvix-stock
Bosch F01U077942
Bosch F01U077942 = 8 505,0 р.; None on auvix-stock
Bosch F01U008873
Bosch F01U008873 = 90 376,0 р.; 1 on auvix-stock
Bosch F01U066788
Bosch F01U066788 = 8 235,0 р.; 20 on auvix-stock
Bosch F01U134980
Bosch F01U134980 = 34 250,0 р.; None on auvix-factory
Bosch F01U077941
Bosch F01U077941 = 8 235,0 р.; 20 on auvix-stock
Bosch F01U100591
Bosch F01U100591 = 13 494,0 р.; None on auvix-factory
Bosch F01U100593
Bosch F01U100593 = 16 335,0 р.; None on auvix-factory
Bosch F01U009512
Bosch F01U009512 = 23 568,0 р.; None on auvix-factory
Bosch F01U009513
Bosch F01U009513 = 23 568,0 р.; None on auvix-factory
Bosch DCN-DISCS-L
Bosch DCN-DISCS-L = 23 568,0 р.; None on auvix-factory
Bosch F01U009515
Bosch F01U009515 = 23 568,0 р.; None on auvix-factory
Bosch F01U008404
Bosch F0

Bosch F01U133186 = 15 135,0 р.; 7 on auvix-stock
Bosch F01U287751
Bosch F01U287751 = 59 067,0 р.; None on auvix-factory
Bosch F01U287752
Bosch F01U287752 = 98 439,0 р.; None on auvix-factory
Projecta 11200038
Projecta 11200038 = 192,0 EUR; 20 on auvix-stock
Projecta 11200039
Projecta 11200039 = 236,0 EUR; 2 on auvix-stock
Projecta 11200040
Projecta 11200040 = 477,0 EUR; 1 on auvix-stock
Projecta 11200042
Projecta 11200042 = 823,0 EUR; 1 on auvix-stock
Projecta 11200036
Projecta 11200036 = 121,0 EUR; 11 on auvix-stock
Projecta 11300001
Projecta 11300001 = 275,0 EUR; None on auvix-factory
Projecta 11300006
Projecta 11300006 = 275,0 EUR; None on auvix-factory
Lumien LTD-101
Lumien LTD-101 = 80,0 $; 20 on auvix-stock
Lumien LTG-102
Lumien LTG-102 = 212,0 $; 25 on auvix-stock
Lumien LTG-102
Lumien LTG-102 = 212,0 $; 22 on auvix-stock
Lumien LTV-103
Lumien LTV-103 = 102,0 $; 2 on auvix-stock
AOpen DE67-HA
AOpen DE67-HA = 1 521,0 $; 2 on auvix-stock
AOpen DE3100
AOpen DE3100 = 703,0 $; 11 on 

Ubiquiti UAP-AC-LITE-EU = 90,0 $; 5 on auvix-stock
Ubiquiti UAP-AC-LR-EU
Ubiquiti UAP-AC-LR-EU = 111,0 $; 5 on auvix-stock
Ubiquiti UAP-AC-LR-5-EU
Ubiquiti UAP-AC-LR-5-EU = 511,0 $; None on auvix-stock
Ubiquiti UAP-AC-PRO-EU
Ubiquiti UAP-AC-PRO-EU = 153,0 $; None on auvix-factory
Ubiquiti UAP-AC-PRO-5-EU
Ubiquiti UAP-AC-PRO-5-EU = 719,0 $; None on auvix-factory
Ubiquiti AM-2G15-120
Ubiquiti AM-2G15-120 = 151,0 $; 2 on auvix-stock
Ubiquiti AM-5G19-120
Ubiquiti AM-5G19-120 = 139,0 $; 1 on auvix-stock
Ubiquiti airGateway
Ubiquiti airGateway = 16,0 $; 20 on auvix-stock
Mikrotik S+31DLC10D
Mikrotik S+31DLC10D = 133,0 $; None on auvix-factory
Mikrotik RBPOE
Mikrotik RBPOE = 3,0 $; None on auvix-factory
Mikrotik RBGPOE
Mikrotik RBGPOE = 6,0 $; None on auvix-factory
Mikrotik RBGPOE-CON-HP
Mikrotik RBGPOE-CON-HP = 24,0 $; 5 on auvix-stock
Mikrotik RB411AH
Mikrotik RB411AH = 93,0 $; 5 on auvix-stock
Mikrotik RB433AH
Mikrotik RB433AH = 140,0 $; 5 on auvix-stock
Mikrotik RB912UAG-2HPnD
Mikrotik RB

Netatmo NOC01-P2 = 320,0 $; None on auvix-factory
Totolink N500UD
Totolink N500UD = 16,0 $; 10 on auvix-stock
ASUS RT-N11P
ASUS RT-N11P = 20,0 $; 30 on auvix-stock
ASUS EA-N66
ASUS EA-N66 = 98,0 $; 5 on auvix-stock
ASUS EA-AC87
ASUS EA-AC87 = 120,0 $; 5 on auvix-stock
ASUS PL-N12kit
ASUS PL-N12kit = 62,0 $; 30 on auvix-stock
ASUS USB-AC56
ASUS USB-AC56 = 64,0 $; 10 on auvix-stock
Ubiquiti INS-8023AF-I
Ubiquiti INS-8023AF-I = 19,0 $; 5 on auvix-stock
Ubiquiti INS-8023AF-O
Ubiquiti INS-8023AF-O = 22,0 $; 5 on auvix-stock
Ubiquiti TS-16-CarrierEU
Ubiquiti TS-16-CarrierEU = 451,0 $; 2 on auvix-stock
Ubiquiti mFi-CS
Ubiquiti mFi-CS = 16,0 $; 5 on auvix-stock
Ubiquiti mFi-DS
Ubiquiti mFi-DS = 10,0 $; 5 on auvix-stock
Ubiquiti mFi-MSC
Ubiquiti mFi-MSC = 48,0 $; 5 on auvix-stock
Ubiquiti mFi-MSW
Ubiquiti mFi-MSW = 35,0 $; 5 on auvix-stock
Ubiquiti mFi-THS
Ubiquiti mFi-THS = 25,0 $; 5 on auvix-stock
Ubiquiti mPower-MiniEU
Ubiquiti mPower-MiniEU = 46,0 $; 5 on auvix-stock
Ubiquiti mPowerEU
Ubiqu

In [15]:
print('Количество продуктов:', len(products))

Количество продуктов: 5868


In [16]:
print('Количество категорий:', len(categories))

Количество категорий: 5868


In [17]:
for n, product in enumerate(products):
    print('{} {}'.format(n, product, categories[n]))

0 InFocus SP1080
1 InFocus IN119HDx
2 InFocus IN128HDx
3 InFocus IN128HDSTx
4 InFocus IN2128HDx
5 InFocus IN112x
6 InFocus IN220
7 InFocus IN114x
8 InFocus IN124a
9 InFocus IN124x
10 InFocus IN2124a
11 InFocus IN2124x
12 InFocus IN116x
13 InFocus IN126a
14 InFocus IN126x
15 InFocus IN2126a
16 InFocus IN2126x
17 InFocus IN124STa
18 InFocus IN124STx
19 InFocus IN126STa
20 InFocus IN126STx
21 InFocus IN134UST
22 InFocus IN136UST
23 InFocus IN1116
24 InFocus IN1118HD
25 InFocus IN5148HD
26 InFocus IN5312a
27 InFocus IN5316HDa
28 InFocus IN5544
29 InFocus IN5552L
30 InFocus IN5554L
31 InFocus IN5555L
32 InFocus LightCast
33 InFocus INA-LCKEY2
34 InFocus SP-WIFIUSB-2
35 InFocus SP-DVI-A-F-R
36 InFocus SP-DVI-D-R
37 InFocus SP-HDMI-ADPT-R
38 InFocus CA-SOFTCASE-MTG
39 InFocus PRJ-WALLKIT-14
40 InFocus HW-NAVIGATOR-4
41 NEC VE303
42 NEC VE281X
43 NEC VE303X
44 NEC V302W
45 NEC V332W
46 NEC ME301X
47 NEC ME331X
48 NEC ME301W
49 NEC ME331W
50 NEC ME361X
51 NEC ME361W
52 NEC M403W
53 NEC M333XS
5

1960 Projecta 10530026
1961 Projecta 10530249
1962 Projecta 10530250
1963 Projecta 10530251
1964 Projecta 10530690
1965 Projecta 10530686
1966 Projecta 10530694
1967 Projecta 10530015
1968 Projecta 10530012
1969 Projecta 10530018
1970 Projecta 10530019
1971 Projecta 10530025
1972 Projecta 10530016
1973 Projecta 10530663
1974 Projecta 10530665
1975 Projecta 10530667
1976 Projecta 10530669
1977 Projecta 10530671
1978 Projecta 10530673
1979 Projecta 10530675
1980 Projecta 10530677
1981 Projecta 10530679
1982 Projecta 10530683
1983 Projecta 10530687
1984 Projecta 10530691
1985 Projecta 10530695
1986 Projecta 10530697
1987 Projecta 10530701
1988 Projecta 10530703
1989 Projecta 10530707
1990 Projecta 10530664
1991 Projecta 10530666
1992 Projecta 10530668
1993 Projecta 10530670
1994 Projecta 10530672
1995 Projecta 10530674
1996 Projecta 10530676
1997 Projecta 10530678
1998 Projecta 10530680
1999 Projecta 10530684
2000 Projecta 10530688
2001 Projecta 10530692
2002 Projecta 10530696
2003 Projec

3422 Комплект T82/IN124STa/WTH140
3423 Комплект N82/IN124STa/WTH140
3424 Комплект N82+acc/IN124STa/WTH140
3425 Комплект T82/IN134UST/WTH140
3426 Комплект N82/IN134UST/WTH140
3427 Комплект N82+acc/IN134UST/WTH140
3428 Current Audio CS804FL
3429 Current Audio CS802FL
3430 Current Audio CS800DTFL
3431 Current Audio CS651FL
3432 Current Audio CS650FL
3433 Current Audio CS654DBFL
3434 Current Audio CS650DT
3435 Current Audio CS650DBFL
3436 Current Audio FIT804FL
3437 Current Audio FIT800FL
3438 Current Audio FIT654FL
3439 Current Audio FIT654
3440 Current Audio FIT652FL
3441 Current Audio FIT652
3442 Current Audio FIT651
3443 Current Audio WSLCR654FL
3444 Current Audio WSLCR650FL
3445 Current Audio WS651
3446 Current Audio WS651FL
3447 Current Audio WS804
3448 Current Audio WS804FL
3449 Current Audio WS801
3450 Current Audio WS801FL
3451 Current Audio WS650
3452 Current Audio WS650FL
3453 Current Audio SPA525FR
3454 Current Audio OC65B
3455 Current Audio OC525B
3456 Current Audio OC65B70V
3

4192 Kramer Electronics SG-6005XL
4193 Kramer Electronics VA-1DVIN
4194 Kramer Electronics VA-2H
4195 Kramer Electronics FC-200
4196 Kramer Electronics 840HXL
4197 Kramer Electronics VA-4H
4198 Kramer Electronics PIP-4
4199 Kramer Electronics 850
4200 Kramer Electronics KDS-MP1
4201 Kramer Electronics TR-1YC
4202 Kramer Electronics KDS-DEC3
4203 Kramer Electronics RK-MEDN
4204 Kramer Electronics RK-81
4205 Kramer Electronics RK-4X
4206 Kramer Electronics RK-701xl
4207 Kramer Electronics RK-WP6G
4208 Kramer Electronics RK-WP16
4209 Kramer Electronics RK-121WP
4210 Kramer Electronics RK-22WP
4211 Kramer Electronics RK-3T
4212 Kramer Electronics RK-6T
4213 Kramer Electronics RK-9T
4214 Kramer Electronics RK-2T1PT
4215 Kramer Electronics RK-4PT
4216 Kramer Electronics RK-3TR
4217 Kramer Electronics RK-T2B
4218 Kramer Electronics RK-T2SB
4219 Kramer Electronics RK-13
4220 Kramer Electronics RK-1
4221 Kramer Electronics RK-19
4222 Extron 70-077-01
4223 Kramer Electronics RK-4E/S
4224 Kramer 